Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [2]:
labels = [
        {'name':'A', 'id':1},
        {'name':'B', 'id':2},
        {'name':'C', 'id':3},
        {'name':'D', 'id':4},
        {'name':'E', 'id':5}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [3]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [4]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.
'cp' is not recognized as an internal or external command,
operable program or batch file.


# 5. Update Config For Transfer Learning

In [1]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [2]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

NameError: name 'MODEL_PATH' is not defined

In [7]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [7]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [8]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [9]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [13]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [8]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [10]:
import cv2 
import numpy as np
import time

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    t = time.time()
    ret, frame = cap.read()
    image_np = np.array(frame)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)
    print([category_index.get(i) for i in detections['detection_classes']])
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    print(num_detections)
    print('fps', 1/(time.time()-t))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'

100
fps 18.579584315254177
[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, None, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, None, {'id': 1, 'name': 'A'}, None, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id':

100
fps 19.128577227059
[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, None, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 4,

100
fps 18.31461832027736
[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {

100
fps 20.05193811791253
[{'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, None, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, None, None, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {

...

[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, None, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, None, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, None, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, None, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name

100
fps 17.590310513160322
[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, None, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, None, {'id': 2, 'name': 'B'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, None, {'id': 1, 'name': 'A'}, 

[{'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, None, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, None, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, Non

100
fps 17.285763151929775
[{'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, None, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, None, {'id': 2, 'name': 'B'}, None, None, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, None, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 1, 'name': 'A'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, None, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 

100
fps 17.282130731450046
[{'id': 4, 'name': 'D'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, None, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, {'id': 3, 'name': 'C'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 3, 'name': 'C'}, {'id': 3, 'name': 'C'}, None, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}, None, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, None, None, {'id': 1, 'name': 'A'}, {'id': 4, 'name': 'D'}, None, {'id': 4, 'name': 'D'}, {'id': 2, 'name': 'B'}, {'id': 1, 'name': 'A'}, 

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt